<a href="https://colab.research.google.com/github/anirudhr20/Brain-Tumor-Detection-Using-Faster-RCNN/blob/master/F_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


4/uAHlg9MeLLSxHAqjzkdzb3qubqIHxsxr9G1PLbmcD5irSjpqzSoyH0E


In [0]:
PATH_OF_DATA= '/content/gdrive/"My Drive"/data/mri-brain'
!ls {PATH_OF_DATA}

100.mat  179.mat  256.mat  333.mat  410.mat  489.mat  566.mat  643.mat	720.mat
101.mat  17.mat   257.mat  334.mat  411.mat  48.mat   567.mat  644.mat	721.mat
102.mat  180.mat  258.mat  335.mat  412.mat  490.mat  568.mat  645.mat	722.mat
103.mat  181.mat  259.mat  336.mat  413.mat  491.mat  569.mat  646.mat	723.mat
104.mat  182.mat  25.mat   337.mat  414.mat  492.mat  56.mat   647.mat	724.mat
105.mat  183.mat  260.mat  338.mat  415.mat  493.mat  570.mat  648.mat	725.mat
106.mat  184.mat  261.mat  339.mat  416.mat  494.mat  571.mat  649.mat	726.mat
107.mat  185.mat  262.mat  33.mat   417.mat  495.mat  572.mat  64.mat	727.mat
108.mat  186.mat  263.mat  340.mat  418.mat  496.mat  573.mat  650.mat	728.mat
109.mat  187.mat  264.mat  341.mat  419.mat  497.mat  574.mat  651.mat	729.mat
10.mat	 188.mat  265.mat  342.mat  41.mat   498.mat  575.mat  652.mat	72.mat
110.mat  189.mat  266.mat  343.mat  420.mat  499.mat  576.mat  653.mat	730.mat
111.mat  18.mat   267.mat  344.mat  421.mat  49.mat   5

In [0]:
import torch
import numpy as np
import torchvision.transforms as transforms
from torchvision import datasets
import os
import shutil

In [0]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [0]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 6019, done.
remote: Total 6019 (delta 0), reused 0 (delta 0), pack-reused 6019
Receiving objects: 100% (6019/6019), 9.57 MiB | 12.70 MiB/s, done.
Resolving deltas: 100% (4004/4004), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be37608 version check against PyTorch's CUDA version


In [0]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

NameError: ignored

In [0]:
fmodel = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = fmodel.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
fmodel.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:04<00:00, 40.1MB/s]


In [0]:
device = torch.device('cpu')
fmodel.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
       

In [0]:

params = [p for p in fmodel.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

data_transforms = transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])

In [0]:
train_data = datasets.ImageFolder(root='/content/gdrive/My Drive/data/Tumor/training_set',transform=data_transform)
test_data = datasets.ImageFolder(root='/content/gdrive/My Drive/data/Tumor/test_set',transform=data_transform)

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
num_train = len(train_data)
valid_size = 0.2
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx = indices[split:]

train_sampler = SubsetRandomSampler(train_idx)



In [0]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

In [0]:
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [0]:
# let's train it for 10 epochs
device = torch.device('cpu')
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(fmodel, optimizer, t_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(fmodel, tes_loader, device=device)

TypeError: ignored

In [0]:
import torch
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.utils.data
from PIL import Image, ImageFile
import pandas as pd
from tqdm import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
def _get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [0]:
import collections
import os
import numpy as np


class OpenDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, height, width, transforms=None):
        self.transforms = transforms
        self.image_dir = image_dir
        # self.df = pd.read_csv(df_path)
        self.height = height
        self.width = width
        self.image_info = collections.defaultdict(dict)
        
        # Filling up image_info is left as an exercise to the reader
        

    def __getitem__(self, idx):
        # load images ad masks
        img_path = self.image_info[idx]["image_path"]
        img = Image.open(img_path).convert("RGB")
        img = img.resize((self.width, self.height), resample=Image.BILINEAR)
        
        # processing part and extraction of boxes is left as an exercise to the reader
        
        boxes = torch.rand(10, 4) * 100
        boxes[:, 2:] += boxes[:, :2]
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        target = {}
        target["boxes"] = boxes
        # target["labels"] = labels
        # target["image_id"] = image_id
        # target["area"] = area
        # target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.image_info)

In [0]:
num_classes = 2
device = torch.device('cuda:0')

dataset_train = OpenDataset('/content/gdrive/My Drive/data/Tumor/training_set', 128, 128, transforms=None)

model_ft = _get_instance_segmentation_model(num_classes)
model_ft.to(device)






FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
       

In [0]:
# data_loader = torch.utils.data.DataLoader(
#     dataset_train, batch_size=4, shuffle=True, num_workers=8,
#     collate_fn=utils.collate_fn)
train = torch.utils.data.DataLoader(
    dataset_train, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

ValueError: ignored

In [0]:
params = [p for p in model_ft.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=5,
                                               gamma=0.1)

In [0]:
num_epochs = 8
for epoch in range(num_epochs):
    train_one_epoch(model_ft, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()